<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-Data" data-toc-modified-id="Get-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get Data</a></span></li><li><span><a href="#Try-various-models" data-toc-modified-id="Try-various-models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Try various models</a></span><ul class="toc-item"><li><span><a href="#Score-on-train-set" data-toc-modified-id="Score-on-train-set-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Score on train set</a></span></li><li><span><a href="#Cross-val-score-on-validation-set" data-toc-modified-id="Cross-val-score-on-validation-set-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Cross val score on validation set</a></span><ul class="toc-item"><li><span><a href="#Stochastic-gradient-descent-classifier" data-toc-modified-id="Stochastic-gradient-descent-classifier-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Stochastic gradient descent classifier</a></span></li><li><span><a href="#Logistic-regression" data-toc-modified-id="Logistic-regression-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Logistic regression</a></span></li><li><span><a href="#RandomForest-Classifier" data-toc-modified-id="RandomForest-Classifier-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>RandomForest Classifier</a></span></li><li><span><a href="#Same-scores-with-better-vizualisation" data-toc-modified-id="Same-scores-with-better-vizualisation-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Same scores with better vizualisation</a></span></li><li><span><a href="#Trying-with-XGBoost" data-toc-modified-id="Trying-with-XGBoost-2.2.5"><span class="toc-item-num">2.2.5&nbsp;&nbsp;</span>Trying with XGBoost</a></span></li></ul></li><li><span><a href="#Finetuning-xgboost" data-toc-modified-id="Finetuning-xgboost-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Finetuning xgboost</a></span></li></ul></li></ul></div>

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

# Get Data

In [2]:
data_train= pd.read_csv("X_train_final_duration.csv")
data_train.drop(['meals_saved.1', 'total_supply.1', 'item_view.1'], axis=1, inplace=True)

In [26]:
X_train = data_train.sample(n=500000, random_state=1)#, no need to use all the data for fine tuning

In [ ]:
stringency = pd.read_excel("stringency.xlsx") #adding oxford economics stringency index regarding government restriction

In [30]:
X_train.shape

(472543, 119)

In [31]:
a = X_train['target'] != 0
X_train['churn_duration'] = a.cumsum()-a.cumsum().where(~a).ffill().fillna(0).astype(int)
# should we try to fit with other stores too ?

X_train = X_train[X_train['churn_duration'] <= 15]
X_train.drop(['churn_duration'], axis=1, inplace=True)
X_train = X_train.dropna()

print(1)

stringency['date'] = pd.to_datetime(stringency['stringency'], format='%d%b%Y')
stringency.drop(columns=['stringency'], axis=1, inplace=True)
X_train['date'] = pd.to_datetime(X_train['date'])
X_train = X_train.merge(stringency, how='left', on='date')
X_train = X_train.fillna(0)

print(2)

y = X_train['target']
X_train.drop(['target', 'date', 'store_id'], axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y, test_size=0.25, random_state=42, shuffle=True)

1
2


In [32]:
X_train

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,weekend,parent_chain_id,total_supply,...,total_supply_lag_13,meals_saved_lag_13,item_view_lag_13,total_supply_lag_14,meals_saved_lag_14,item_view_lag_14,total_supply_lag_15,meals_saved_lag_15,item_view_lag_15,FRA
173587,0,0,1,0,0,0,0,0,4987.0,2,...,5.0,5.0,60.0,0.0,0.0,46.0,1.0,1.0,89.0,87.96
127952,0,1,0,0,0,0,0,0,0.0,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
214760,0,0,0,1,0,0,0,0,0.0,6,...,6.0,6.0,0.0,6.0,6.0,0.0,6.0,6.0,0.0,0.00
170074,1,0,0,0,0,0,0,0,4597.0,0,...,2.0,2.0,12.0,0.0,0.0,20.0,2.0,2.0,8.0,56.94
414125,0,1,0,0,0,0,0,0,4329.0,3,...,3.0,2.0,112.0,3.0,3.0,148.0,3.0,3.0,107.0,13.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,0,1,0,0,0,0,0,0,3534.0,0,...,10.0,10.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.00
365838,0,0,1,0,0,0,0,0,0.0,3,...,4.0,4.0,0.0,3.0,3.0,0.0,2.0,2.0,0.0,0.00
131932,0,0,0,1,0,0,0,0,0.0,12,...,2.0,2.0,0.0,16.0,16.0,0.0,2.0,2.0,0.0,0.00
146867,0,0,0,0,0,0,1,1,211.0,0,...,0.0,0.0,16.0,0.0,0.0,7.0,0.0,0.0,5.0,0.00


# Try various models

In [33]:

from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
scaler = preprocessing.StandardScaler().fit(X_train)

recall = make_scorer(recall_score) # Pour passer le score en argument dans cross_val_score
precision = make_scorer(precision_score)
f1 = make_scorer(f1_score)
accuracy = make_scorer(accuracy_score)

X_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import cross_val_score

## Score on train set

In [ ]:
sgd_clf = SGDClassifier(random_state=1, loss='log', class_weight='balanced')
lreg_clf = LogisticRegression(max_iter=1000, random_state=0, class_weight='balanced')
rnd_clf=RandomForestClassifier(n_estimators=500, max_depth=7, random_state=0, class_weight='balanced_subsample')

def get_score_train(model,X_prepared=X_scaled,y_train=y_train): 
    '''Fonction qui permet de calculer l'ensemble des scores étant données un modèle et un jeu de données 
    (features + etiquette
    La fonction entraine le modèle et utilise les prédictions à partir du train pour évaluer les scores'''

    model.fit(X_prepared, y_train)
    y_pred = model.predict(X_prepared)
    print('Recall train :' + '{:.2%}'.format(recall_score( y_train,y_pred)))
    print('Precision train :' + '{:.2%}'.format(precision_score(y_train,y_pred )))
    print('F1 train :' + '{:.2%}'.format(f1_score(y_train, y_pred)))
    print('Accuracy :' + '{:.2%}'.format(accuracy_score(y_train, y_pred)))

In [108]:
get_score_train(lreg_clf, X_scaled, y_train)

Recall train :95.94%
Precision train :48.77%
F1 train :64.67%


In [110]:
get_score_train(sgd_clf,X_scaled, y_train)

Recall train :96.20%
Precision train :46.27%
F1 train :62.49%


In [111]:
get_score_train(rnd_clf,X_scaled, y_train)

Recall train :95.16%
Precision train :59.47%
F1 train :73.19%


## Cross val score on validation set

### Stochastic gradient descent classifier 

In [112]:
#On utilise maintenant de la cross-validation pour évaluer l'erreur sur des validation sets (5 folds ici) 


scores_recall = cross_val_score(sgd_clf, X_test_scaled, y_test, cv=5, scoring=recall)
scores_precision= cross_val_score(sgd_clf, X_test_scaled, y_test, cv=5, scoring=precision)
scores_f1= cross_val_score(sgd_clf, X_test_scaled, y_test, cv=5, scoring=f1)

#On conservera la moyenne des scores

def display_scores(scores):
    print("Scores :", scores)
    print("Moyenne :" + '{:.2%}'.format(scores.mean()))
    print("Écart type :" + '{:.2%}'.format(scores.std()))

print('Recall :')
print(display_scores(scores_recall))

print('\n'+'Precision')
display_scores(scores_precision)

print('\n'+'F1 score')
display_scores(scores_f1)

Recall :
Scores : [0.92774566 0.93641618 0.90303907 0.95658466 0.95947902]
Moyenne :93.67%
Écart type :2.06%
None

Precision
Scores : [0.5043205  0.4733382  0.47742923 0.44571814 0.46009715]
Moyenne :47.22%
Écart type :1.95%

F1 score
Scores : [0.65343511 0.62882096 0.62462462 0.60809568 0.62195122]
Moyenne :62.74%
Écart type :1.48%


### Logistic regression

In [113]:
scores_recall = cross_val_score(lreg_clf, X_test_scaled, y_test, cv=5, scoring=recall)
scores_precision= cross_val_score(lreg_clf, X_test_scaled, y_test, cv=5, scoring=precision)
scores_f1= cross_val_score(lreg_clf, X_test_scaled, y_test, cv=5, scoring=f1)

print('Recall :')
print(display_scores(scores_recall))

print('\n'+'Precision')
display_scores(scores_precision)

print('\n'+'F1 score')
display_scores(scores_f1)

Recall :
Scores : [0.96965318 0.94942197 0.95513748 0.95079595 0.94356006]
Moyenne :95.37%
Écart type :0.88%
None

Precision
Scores : [0.51378254 0.47886297 0.47447879 0.49213483 0.48948949]
Moyenne :48.97%
Écart type :1.37%

F1 score
Scores : [0.67167167 0.63662791 0.63400576 0.64856861 0.64458725]
Moyenne :64.71%
Écart type :1.34%


### RandomForest Classifier

In [114]:
scores_recall = cross_val_score(rnd_clf, X_test_scaled, y_test, cv=5, scoring=recall)
scores_precision= cross_val_score(rnd_clf, X_test_scaled, y_test, cv=5, scoring=precision)
scores_f1= cross_val_score(rnd_clf, X_test_scaled, y_test, cv=5, scoring=f1)
print('Recall :')
print(display_scores(scores_recall))

print('\n'+'Precision')
display_scores(scores_precision)

print('\n'+'F1 score')
display_scores(scores_f1)

Recall :
Scores : [0.94075145 0.92196532 0.94356006 0.91606368 0.91751085]
Moyenne :92.80%
Écart type :1.18%
None

Precision
Scores : [0.62959381 0.59074074 0.60764212 0.60285714 0.60727969]
Moyenne :60.76%
Écart type :1.26%

F1 score
Scores : [0.75434531 0.72009029 0.73922902 0.72716829 0.73083573]
Moyenne :73.43%
Écart type :1.18%


### Same scores with better vizualisation 

In [ ]:
#On définit une fonction pour obtenir les listes de scores calculées par cross_val sur chacun des modèles
def display_scores_df(model, X_prepared=X_scaled,y_train=y_train,cv=5):
    '''Fonction prenant en argument un modèle, un jeu de données (features +etiquette) 
    et le nombre de Folds pour la cross_val'''
    
    scores_recall = cross_val_score(model, X_prepared, y_train, cv=cv, scoring=recall)
    scores_precision= cross_val_score(model, X_prepared, y_train, cv=cv, scoring=precision)
    scores_f1= cross_val_score(model, X_prepared, y_train, cv=cv, scoring=f1)
    scores_accuracy = cross_val_score(model, X_prepared, y_train, cv=cv, scoring=accuracy)
    df_score=pd.DataFrame({'Moyennes' : [scores_recall.mean(), scores_precision.mean(), scores_f1.mean(),scores_accuracy.mean()],
                          'Ecart Type': [scores_recall.std(), scores_precision.std(), scores_f1.std(), scores_accuracy.std()] },
                        index=['Recall', 'Precision', 'F1 Score', 'Accuracy'])
    return df_score

In [130]:
resultats_sgd_clf=display_scores_df(sgd_clf,X_scaled,y_train,cv=5) 
resultats_sgd_clf

,Moyennes,Ecart Type
Recall,0.944511,0.014536
Precision,0.487020,0.010625
F1 Score,0.642529,0.008737


### Trying with XGBoost

In [129]:
xgb_clf = xgb.XGBClassifier( verbosity=0)
print(get_score_train(xgb_clf,X_scaled,y_train))
resultats_xgb_clf=display_scores_df(xgb_clf,X_test_scaled,y_test,cv=5) 
resultats_xgb_clf

C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Recall train :90.79%
Precision train :84.57%
F1 train :87.57%
None


C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py

,Moyennes,Ecart Type
Recall,0.821807,0.010218
Precision,0.776908,0.009240
F1 Score,0.798703,0.008650


XGBoost is the best model so far (regarding F1 score, which might be the most interesting metric for TGTG's business). 

## Finetuning xgboost

In [13]:
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import GridSearchCV

In [ ]:
random_search=RandomizedSearchCV(xgb_clf, param_distributions={'n_estimators': [200, 400, 600, 800,1000],
                                              'max_features': ['auto', 'sqrt', 'log2'],'max_depth' : [10, 20, 30, 40],
                                              'learning_rate': [0.1, 0.05, 0.01],'criterion' :['gini', 'entropy'],
                                              'bootstrap' : [True, False]},
                                               verbose=2, random_state=1, n_jobs = -1, cv=3, scoring=f1)

In [20]:

# Grid search was too long, let's do it the dirty way with fewer parameters

n_estimators=[200, 400, 600, 800] #400
max_depths =[3, 4, 6, 8] #4
learning_rate=[0.1, 0.05, 0.01] # 0.05
for n in tqdm(n_estimators):
    print('estimators', n)
    for depth in tqdm(max_depths):
        print('depth:' , depth)
        for lr in tqdm(learning_rate):
            print('learning rate',lr)
            xgb_clf = xgb.XGBClassifier(n_estimators= n , max_depth = depth ,learning_rate=lr, verbosity =0)
            resultats_xgb_clf=display_scores_df(xgb_clf,X_test_scaled,y_test,cv=3) 
            print(resultats_xgb_clf)
#okay too many outputs, should have save it in dataframes for vizu.. 

estimators 200


depth: 3


learning rate 0.1


C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py

           Moyennes  Ecart Type
Recall     0.853345    0.011652
Precision  0.752394    0.008926
F1 Score   0.799670    0.009089
learning rate 0.05


...

learning rate 0.1


C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py

           Moyennes  Ecart Type
Recall     0.800119    0.010457
Precision  0.785113    0.004739
F1 Score   0.792525    0.006987
learning rate 0.05


C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py

KeyboardInterrupt: 

In [21]:
#the two lines for our best modelso far
import xgboost as xgb
xgb_best_clf= xgb.XGBClassifier(n_estimators= 400 , max_depth = 4 ,learning_rate= 0.05, verbosity =0)

In [22]:
resultats_xgb_best_clf=display_scores_df(xgb_best_clf,X_scaled,y_train,cv=5) 
resultats_xgb_best_clf #doing pretty good on validation set 

C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\xgboost\sklearn.py

,Moyennes,Ecart Type
Recall,0.857913,0.013713
Precision,0.769761,0.003838
F1 Score,0.811396,0.006719
